# UN Speeches – Scraper

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import urllib.request

## Scrape list of all speakers, and links to their speeches:

In [2]:
base_url = requests.get("https://gadebate.un.org/en/sessions-archive/72")
print(base_url.status_code)
soup = BeautifulSoup(base_url.content, 'html.parser')

200


In [3]:
soup = soup.findAll(class_ = "media-body")

In [4]:
columns = ['Country','Name', 'Title', 'link', 'speech']
df = pd.DataFrame(index=np.arange(len(soup)), columns=columns)

In [5]:
for i in range(0, len(soup)):
    df['Country'][i] = soup[i].find(class_ = "media-heading").text
    df['Name'][i] = soup[i].find(class_ = "name").text
    df['Title'][i] = soup[i].find(class_ = "title").text
    df['link'][i] = 'https://gadebate.un.org'+soup[i].find('a').get('href')

In [6]:
for i in range(0, len(soup)):
    print("Scraping: {}/{}".format(i+1, len(soup)), end = '\r')
    temp_url = requests.get(df['link'][i])
    temp_soup = BeautifulSoup(temp_url.content, 'html.parser')
    temp_soup = temp_soup.find(class_ = "statement-documents-list")
    temp_soup = temp_soup.findAll('a')
    for j in range(0, len(temp_soup)):
        if temp_soup[j].text == "English":
            df['speech'][i] = temp_soup[j].get('href')
            break
print("\nDone!")

Scraping: 199/199
Done!


In [7]:
df.head()

,Country,Name,Title,link,speech
0,Secretary-General of the United Nations,H.E. Mr. António Guterres,Secretary-General,https://gadebate.un.org/en/72/secretary-genera...,https://gadebate.un.org/sites/default/files/ga...
1,President of the General Assembly (opening),H.E. Mr. Miroslav Lajčák,President of the 72nd Session of the General A...,https://gadebate.un.org/en/72/president-genera...,https://gadebate.un.org/sites/default/files/ga...
2,Brazil,H.E. Mr. Michel Temer,President,https://gadebate.un.org/en/72/brazil,https://gadebate.un.org/sites/default/files/ga...
3,United States of America,H.E. Mr. Donald J. Trump,President,https://gadebate.un.org/en/72/united-states-am...,https://gadebate.un.org/sites/default/files/ga...
4,Guinea,H.E. Mr. Alpha Condé,President,https://gadebate.un.org/en/72/guinea,NaN


## Download PDF versions of speeches:

In [8]:
for i in range(0, len(soup)):
    print("Downloading PDF versions: {}/{}".format(i+1, len(soup)), end = '\r')
    if pd.isnull(df['speech'][i])==False:
        url = df['speech'][i]
        urllib.request.urlretrieve(url, '../data/raw_data/{}.pdf'.format(df['Country'][i]))
print("\nDone!")

Done!


## Converting to .txt

Conversion to .txt is done using pdftotxt from the xpdf suite.